In [208]:
import pandas as pd
import geopandas as gpd
from datetime import datetime
import json
import numpy as np
import locale
import os
# Imposta la lingua italiana
locale.setlocale(locale.LC_TIME, 'it_IT.UTF-8')

'it_IT.UTF-8'

In [209]:
PARKS_GEOPARQUET = "data" + os.sep + "parks.geoparquet"
ZONES_GEOPARQUET = "data" + os.sep + "zones.geoparquet"


In [210]:
parks = gpd.read_parquet(PARKS_GEOPARQUET)
zones = gpd.read_parquet(ZONES_GEOPARQUET)

In [211]:
descriptions = {}

In [212]:
zones.columns

Index(['id', 'driver', 'geom', 'name', 'type', 'city', 'color', 'capacity',
       'freeslots', 'ts', 'stall_blu_capacity', 'stall_blu_freeslots',
       'stall_blu_ts', 'stall_carico-scarico_capacity',
       'stall_carico-scarico_freeslots', 'stall_carico-scarico_ts',
       'stall_disabili_capacity', 'stall_disabili_freeslots',
       'stall_disabili_ts'],
      dtype='object')

In [213]:
# Assuming parks.currentTimestamp contains the timestamp
timestamp_parks = parks.currentTimestamp.max()
lastime_parks = timestamp_parks.strftime('%d/%m/%Y %H:%M')
lastime_parks = timestamp_parks.strftime('%d %B %Y ore %H:%M')

In [214]:
descriptions['timestamp_parks'] = lastime_parks

In [215]:
# Assuming parks.currentTimestamp contains the timestamp
timestamp_zones = zones.ts.max()
lastime_zones = timestamp_zones.strftime('%d/%m/%Y %H:%M')

In [216]:
total_carparkspaces = parks[(parks['type'] == 'park') & (parks.currentTimestamp == timestamp_parks)].capacity.sum()
total_carparkspaces_free = parks[(parks['type'] == 'park') & (parks.currentTimestamp == timestamp_parks)].freeslots.sum()
total_bikeparkspaces = parks[(parks['type'] == 'bike') & (parks.currentTimestamp == timestamp_parks)].capacity.sum()
total_bikeparkspaces_free = parks[(parks['type'] == 'bike') & (parks.currentTimestamp == timestamp_parks)].freeslots.sum()

In [217]:
descriptions["total_carparkspaces"] = total_carparkspaces
descriptions["total_carparkspaces_free"] = total_carparkspaces_free
descriptions["total_bikespaces"] = total_bikeparkspaces
descriptions["total_bikespaces_free"] = total_bikeparkspaces_free

In [218]:
to_int = ['stall_blu_capacity','stall_blu_freeslots',
          'stall_carico-scarico_capacity','stall_carico-scarico_freeslots',
          'stall_disabili_capacity','stall_disabili_freeslots']
zones[to_int] = zones[to_int].astype(int)

In [219]:
total_zonespaces = zones[zones.ts == timestamp_zones].capacity.sum()
total_zonespaces_free = zones[zones.ts == timestamp_zones].freeslots.sum()
total_zonespaces_blu = zones[zones.ts == timestamp_zones].stall_blu_capacity.sum()
total_zonespaces_blu_free = zones[zones.ts == timestamp_zones].stall_blu_freeslots.sum()
total_zonespaces_carico_scarico = zones[zones.ts == timestamp_zones]['stall_carico-scarico_capacity'].sum()
total_zonespaces_carico_scarico_free = zones[zones.ts == timestamp_zones]['stall_carico-scarico_freeslots'].sum()
total_zonespaces_disabili = zones[zones.ts == timestamp_zones].stall_disabili_capacity.sum()
total_zonespaces_disabili_free = zones[zones.ts == timestamp_zones].stall_disabili_freeslots.sum()

In [220]:
descriptions["total_zonespaces"] = total_zonespaces
descriptions["total_zonespaces_free"] = total_zonespaces_free
descriptions["total_zonespaces_blu"] = total_zonespaces_blu
descriptions["total_zonespaces_blu_free"] = total_zonespaces_blu_free
descriptions["total_zonespaces_carico_scarico"] = total_zonespaces_carico_scarico
descriptions["total_zonespaces_carico_scarico_free"] = total_zonespaces_carico_scarico_free
descriptions["total_zonespaces_disabili"] = total_zonespaces_disabili
descriptions["total_zonespaces_disabili_free"] = total_zonespaces_disabili_free

In [221]:
total_parks = len(parks[parks['type'] == 'park'].name.unique())
total_ciclobox = len(parks[parks['type'] == 'bike'].name.unique())

In [222]:
descriptions["total_parks"] = total_parks
descriptions["total_ciclobox"] = total_ciclobox

In [223]:
total_zones = len(zones.name.unique())

In [ ]:
descriptions["total_zones"] = total_zones

In [225]:
zones[zones.stall_disabili_capacity > 0].name.unique()
zones[zones["stall_carico-scarico_capacity"] > 0].name.unique()

array(['Area di 1a corona'], dtype=object)

In [226]:
descriptions = {k: int(v) if isinstance(v, np.integer) else v for k, v in descriptions.items()}

In [227]:
# Scrittura su file JSON
with open("docs/data/descriptions.json", "w") as f:
    json.dump(descriptions, f, indent=4)